In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard

In [ ]:
# Enable mixed precision for faster training (if supported by hardware)
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy) # set_policy is deprecated, use set_global_policy

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    brightness_range=[0.8, 1.2],
    zoom_range=0.3,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    validation_split=0.2
)

# Rescaling
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Training and Validation Dataset
train_data = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/dataset/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_data = val_datagen.flow_from_directory(
    "/content/drive/MyDrive/dataset/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

Found 558 images belonging to 2 classes.
Found 174 images belonging to 2 classes.


In [ ]:
# Indices
print("Class Indices (Train):", train_data.class_indices)
print("Class Indices (Validation):", val_data.class_indices)
print(f"Training samples: {train_data.samples}")
print(f"Validation samples: {val_data.samples}")

Class Indices (Train): {'KTP': 0, 'non_KTP': 1}
Class Indices (Validation): {'KTP': 0, 'non_KTP': 1}
Training samples: 558
Validation samples: 174


In [ ]:
# Using MobileNetV2 as Base Model
base_model = MobileNetV2(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(1, activation="sigmoid")
])


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [ ]:
# Callbacks
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1
)

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=8,
    restore_best_weights=True
)

In [ ]:
model_checkpoint = ModelCheckpoint(
    filepath="model_best.keras",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [ ]:
tensorboard = TensorBoard(log_dir='/content/logs', histogram_freq=1)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras.src.trainers.data_adapters.py_dataset_adapter")


In [ ]:
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr, model_checkpoint, tensorboard]
)

Epoch 1/50
17/18 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6883 - loss: 0.6696 - precision_1: 0.7072 - recall_1: 0.6606
Epoch 1: val_loss improved from inf to 0.14043, saving model to model_best.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 162s 7s/step - accuracy: 0.7013 - loss: 0.6477 - precision_1: 0.7181 - recall_1: 0.6767 - val_accuracy: 0.9713 - val_loss: 0.1404 - val_precision_1: 0.9767 - val_recall_1: 0.9655 - learning_rate: 0.0010
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step - accuracy: 0.9225 - loss: 0.2499 - precision_1: 0.9363 - recall_1: 0.9074
Epoch 2: val_loss improved from 0.14043 to 0.11467, saving model to model_best.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 616ms/step - accuracy: 0.9227 - loss: 0.2493 - precision_1: 0.9368 - recall_1: 0.9072 - val_accuracy: 0.9770 - val_loss: 0.1147 - val_precision_1: 0.9882 - val_recall_1: 0.9655 - learning_rate: 0.0010
Epoch 3/50
16/18 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step - accuracy: 0.9459 - loss: 0.1988 - precision_1: 0.9438 - recall_1: 0

In [ ]:
# Unfreeze some layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Freeze earlier layers
    layer.trainable = False

In [ ]:
# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9),
    loss="binary_crossentropy",
    metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [ ]:
# Fine-tune the model
history_finetune = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr, model_checkpoint, tensorboard]
)

Epoch 1/50
17/18 ━━━━━━━━━━━━━━━━━━━━ 0s 846ms/step - accuracy: 0.9050 - loss: 0.2642 - precision_2: 0.8752 - recall_2: 0.9452
Epoch 1: val_loss did not improve from 0.07608
18/18 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.9064 - loss: 0.2630 - precision_2: 0.8770 - recall_2: 0.9457 - val_accuracy: 0.9828 - val_loss: 0.0800 - val_precision_2: 0.9884 - val_recall_2: 0.9770 - learning_rate: 1.0000e-04
Epoch 2/50
17/18 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9085 - loss: 0.2495 - precision_2: 0.8870 - recall_2: 0.9319
Epoch 2: val_loss did not improve from 0.07608
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 622ms/step - accuracy: 0.9098 - loss: 0.2472 - precision_2: 0.8890 - recall_2: 0.9326 - val_accuracy: 0.9713 - val_loss: 0.0915 - val_precision_2: 0.9881 - val_recall_2: 0.9540 - learning_rate: 1.0000e-04
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 0.9292 - loss: 0.2298 - precision_2: 0.9594 - recall_2: 0.9040
Epoch 3: val_loss did not improve from 0.07608
18/18 ━━━

In [ ]:
# Evaluate the model
loss, accuracy, precision, recall = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.9794 - loss: 0.0808 - precision_2: 0.6895 - recall_2: 0.6810
Validation Loss: 0.0800
Validation Accuracy: 0.9828
Validation Precision: 0.9884
Validation Recall: 0.9770


In [ ]:
# Generate predictions and evaluate
val_preds = (model.predict(val_data) > 0.5).astype("int32").flatten()
true_labels = val_data.classes


6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 995ms/step


In [ ]:
print("\nClassification Report:")
print(classification_report(true_labels, val_preds))
print("\nConfusion Matrix:")
print(confusion_matrix(true_labels, val_preds))


Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98        87
           1       0.99      0.98      0.98        87

    accuracy                           0.98       174
   macro avg       0.98      0.98      0.98       174
weighted avg       0.98      0.98      0.98       174


Confusion Matrix:
[[86  1]
 [ 2 85]]


In [ ]:
model.save("/content/drive/MyDrive/model_best.keras")